In [31]:
import pandas as pd

In [32]:
df =pd.read_csv('data/data.csv',sep=';')

In [33]:
df.columns = df.columns.str.lower()

In [34]:
df.columns
df.drop(columns=['renaes','ipress','provincia',	'distrito',	'ubigeo','codigo_servicio', 'fecha_internamiento', 'fecha_alta', 'monto_bruto','grupo_cobertura','periodo','codigo_diagnostico','diagnosticos'],inplace=True)

In [42]:
df

,id,departamento,grupo_diagnosticos,sexo,edad,tipo_seguro,servicio,fecha_atencion
0,1,Lima,CANCER DE CUELLO UTERINO,FEMENINO,62,SIS GRATUITO,Teleorientación,20221201
1,1,Lima,CANCER DE CUELLO UTERINO,FEMENINO,61,SIS GRATUITO,Consulta externa,20221115
2,2,Lima,CANCER DE ESTOMAGO,MASCULINO,73,SIS INDEPENDIENTE,Apoyo al diagnóstico (*),20220628
3,2,Lima,CANCER DE ESTOMAGO,MASCULINO,73,SIS INDEPENDIENTE,Consulta externa,20221111
4,2,Lima,CANCER DE ESTOMAGO,MASCULINO,73,SIS INDEPENDIENTE,Consulta externa,20221130
...,...,...,...,...,...,...,...,...
513525,52596,Lima,LEUCEMIA,FEMENINO,0,SIS GRATUITO,Internamiento con intervención quirúrgica mayor,20220914
513526,52596,Lima,LEUCEMIA,FEMENINO,0,SIS GRATUITO,Atención por Telesalud,20220913
513527,52596,Lima,LEUCEMIA,FEMENINO,0,SIS GRATUITO,Consulta externa,20220914
513528,52597,Ucayali,LINFOMA,MASCULINO,0,SIS GRATUITO,Atención por emergencia con observación,20221004
